In [1]:
import xlrd
import numpy as np
import pandas
from pandas import DataFrame

# import data from excel 
if __name__ == '__main__' :
    book = xlrd.open_workbook ('C:/research/python/data/PUM1_results.xlsx')
    sheet = book.sheet_by_name('Sheet1')
    data = []
    data = [[sheet.cell_value(r, col)
                for col in range(sheet.ncols)]
                for r in range(sheet.nrows)]

# numbers are registered as numbers, while leaving the strings alone
for i in range (len (data)):
    for j in range(len(data[0])):
        try:
            data[i][j] = float (data[i][j])
        except:
            pass
        
# Wrap the data in a pandas DataFrame
df = DataFrame(data[1:], columns = data[0])

# extract the significant increased genes by siPUM1
df_q = df[df['q_value'] < 0.05]
df_qup = df_q[df_q['log2_fold_RPKM'] > 0]
df_p = df_qup[df_qup['p_value'] < 0.05]
df_up = df_p[df_p['log2_fold_thalf'] > 0]

# Define the data set of which genes contain PUM motifs
df_up_motif = df_up[df_up['number'] > 0]

# Define the basic transcripts set : RPKM is larger than 0.1 & t_half > 0
df_rpkm = df[df['RPKM_siSte'] > 0.1]
df_base = df_rpkm[df_rpkm['thalf_siSte'] > 0]

In [2]:
df_base.head(3)

,gr_id,symbol,locus,number,sequence,position,3UTR length,RPKM_siSte,RPKM_siPUM1,log2_fold_RPKM,...,thalf_siSte,thalf_siPUM1,log2_fold_thalf,p_value,miRNA_binding_sites,start_site,miRNA,miR_symbol,miR_seq,miR_ID
5,6,A4GALT,chr22:43088126-43116876,0,NaN,NaN,769,3.14619,4.4381,0.496339,...,2.591055,3.352392,0.371651,0.572913,0,NaN,NaN,NaN,NaN,NaN
7,8,AAAS,chr12:53701239-53715412,0,NaN,NaN,33,61.83710,46.6591,-0.406315,...,15.349410,24.000000,0.644851,0.002161,0,NaN,NaN,NaN,NaN,NaN
8,9,AACS,chr12:125549924-125627871,1,TGTATACA,1066,1096,18.12060,18.5598,0.034543,...,8.682658,10.068060,0.213577,0.576386,1,1057,miR-137,hsa-miR-137,UUAUUGCUUAAGAAUACGCGUAG,MIMAT0000429


In [21]:
import matplotlib.pyplot as plt

plt.figure();
plt.subplot(3,1,1)
df_up['RPKM_siPUM1'].plot(kind = 'hist', normed = True, alpha = 0.6, color = 'r', label = 'siPUM1', bins =40, range = (0, 200))
plt.legend(prop={'size':15})
plt.ylabel('Frequency')
#plt.xlim(0, 20)
plt.ylim(0, 0.1)

plt.subplot(3,1,2)
df_up['RPKM_siSte'].plot(kind = 'hist', normed = True, alpha = 0.6, color = 'b', label ='siCTRL', bins =40, range = (0, 200))
plt.legend(prop={'size':15})
plt.xlabel('relative expression')
plt.ylabel('Frequency')
#plt.xlim(0, 20)
plt.ylim(0, 0.1)

plt.subplot(3,1,3)
df_base['RPKM_siSte'].plot(kind = 'hist', normed = True, alpha = 0.6, color = 'k', label ='ALL', bins =40, range = (0, 200))
plt.legend(prop={'size':15})
plt.xlabel('relative expression')
plt.ylabel('Frequency')
#plt.xlim(0, 20)
plt.ylim(0, 0.1)

plt.show()

In [7]:
import numpy as np

In [12]:
plt.figure();
plt.subplot(2,1,1)
df_up['RPKM_siPUM1'].plot(kind = 'hist', normed = True, alpha = 0.6, color = 'r', label = 'siPUM1',bins = 10 ** np.linspace(np.log10(0.1), np.log10(600), 50))
plt.legend(prop={'size':15})
plt.ylabel('Frequency')
plt.gca().set_xscale("log")
plt.ylim(0, 0.3)

plt.subplot(2,1,2)
df_up['RPKM_siSte'].plot(kind = 'hist', normed = True, alpha = 0.6, color = 'b', label ='siSte', bins = 10 ** np.linspace(np.log10(0.1), np.log10(600), 50))
plt.legend(prop={'size':15})
plt.xlabel('Expression')
plt.ylabel('Frequency')
plt.gca().set_xscale("log")
#plt.ylim(0, 0.0015)

plt.show()

In [27]:
# plott a scatter : RPKM_siPUM1 vs RPKM_siSte (log scale)

plt.scatter(df_base['RPKM_siSte'], df_base['RPKM_siPUM1'], marker = 'o', color = 'k', alpha = 0.1)
plt.scatter(df_up['RPKM_siSte'], df_up['RPKM_siPUM1'], marker = 'o', color = 'r', alpha = 0.5)
plt.xlabel('Expression siCTRL (RPKM)')
plt.ylabel('Expression siPUM1 (RPKM)')
plt.gca().set_yscale('log')
plt.gca().set_xscale('log')
plt.show()

In [45]:
# plot a boxplot RPKM

df_up_rpkm = df_up.loc[:, ['RPKM_siSte','RPKM_siPUM1']]

fig, ax = plt.subplots()
plot = df_up_rpkm.boxplot(ax = ax, positions=[1,2], notch = True)


#plt.figure()
#plot = plt.boxplot(df_up_rpkm, widths = 0.7, notch = True, positions = locations, patch_artist = True)
# draw horizontal grid lines
#plt.grid(axis = 'y', linestyle = '--', which = 'major', color = 'lightgrey', alpha = 0.7)
# fill each box in a different color
#for box, colour in zip(plot['boxes'], box_colours):
#    plt.setp(box, color = 'K', linewidth = 1.5, facecolor = colour)
# customize the whiskers, caps, fliers and medians
plt.setp(plot['boxes'], color='darkblue', linewidth = 1.5)
plt.setp(plot['whiskers'], color='K', linewidth = 1.5)
plt.setp(plot['medians'], color='OrangeRed', linewidth = 2.5)
plt.setp(plot['fliers'], color='b', marker='o', markersize=5, alpha = 0.5)

plt.ylabel('Relatice Expression (log)')   
# log scale
plt.gca().set_yscale('log')

plt.show()
    

In [47]:
# plott a scatter : Thalf_siPUM1 vs Thalf_siSte (log scale)

plt.scatter(df_base['thalf_siSte'], df_base['thalf_siPUM1'], marker = 'o', color = 'k', alpha = 0.1)
plt.scatter(df_up['thalf_siSte'], df_up['thalf_siPUM1'], marker = 'o', color = 'r', alpha = 0.5)
plt.xlim(0.1, 25)
plt.ylim(0.1, 25)
plt.xlabel('Half time siCTRL (hr)')
plt.ylabel('Half time siPUM1 (hr)')
plt.gca().set_yscale('log')
plt.gca().set_xscale('log')
plt.show()

In [48]:
# plot a boxplot Thalf

df_up_t = df_up.loc[:, ['thalf_siSte','thalf_siPUM1']]

fig, ax = plt.subplots()
plot = df_up_t.boxplot(ax = ax, positions=[1,2], notch = True)

# customize the whiskers, caps, fliers and medians
plt.setp(plot['boxes'], color='darkblue', linewidth = 1.5)
plt.setp(plot['whiskers'], color='K', linewidth = 1.5)
plt.setp(plot['medians'], color='OrangeRed', linewidth = 2.5)
plt.setp(plot['fliers'], color='b', marker='o', markersize=5, alpha = 0.5)

plt.ylabel('Half time (log)')   
# log scale
plt.gca().set_yscale('log')

plt.show()

In [58]:
plt.figure(figsize = (8,8))
plt.scatter(df_up['log2_fold_thalf'], df_up['log2_fold_RPKM'], marker = 'o', color = 'r', alpha = 0.5, s = 40)
plt.ylabel('Fold change in expression')
plt.xlabel('Fold change in half time')
plt.xlim(0, 4)
plt.ylim(0, 4)
plt.show()

In [59]:
df_fold = df_up.loc[:, ['log2_fold_thalf', 'log2_fold_RPKM']]
df_fold.corr()

,log2_fold_thalf,log2_fold_RPKM
log2_fold_thalf,1.000000,0.504682
log2_fold_RPKM,0.504682,1.000000


In [60]:
from scipy import stats

In [61]:
# statistical test between siSte vs siPUM1
# calculating F value

f = np.var(df_up['RPKM_siSte'] / df_up['RPKM_siPUM1'])
dfx = len(df_up['RPKM_siSte']) - 1
dfy = len(df_up['RPKM_siPUM1']) - 1

p_value = stats.f.cdf(f, dfx, dfy)
print f, dfx, dfy, p_value

0.0149235371228 120 120 2.27544378231e-76


In [62]:
# equal 
t, p = stats.ttest_rel(df_up['RPKM_siSte'], df_up['RPKM_siPUM1'])
print p

5.46199954986e-09


In [63]:
# statistical test between siSte vs siPUM1
# calculating F value

f = np.var(df_up['thalf_siSte'] / df_up['thalf_siPUM1'])
dfx = len(df_up['thalf_siSte']) - 1
dfy = len(df_up['thalf_siPUM1']) - 1

p_value = stats.f.cdf(f, dfx, dfy)
print f, dfx, dfy, p_value

0.0221342882317 120 120 1.84455958903e-66


In [64]:
# equal 
t, p = stats.ttest_rel(df_up['thalf_siSte'], df_up['thalf_siPUM1'])
print p

2.95033887064e-27
